# EDA for Traffic-IMC Dataset

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from metr.components.metr_imc.traffic_data import TrafficData
from metr.utils import PathConfig
import geopandas as gpd
import os
import pickle
import networkx as nx
import numpy as np

In [ ]:
PATH_CONF = PathConfig.from_yaml("../config.yaml")
FILTERED_PATH_CONF = PathConfig.from_yaml("../config_knn.yaml")

TRAFFIC_DATA_PATH = PATH_CONF.metr_imc_path
NODELINK_SHAPE_PATH = PATH_CONF.nodelink_link_path
ADJ_MX_PATH = PATH_CONF.adj_mx_path

FILTERED_DATA_PATH = FILTERED_PATH_CONF.metr_imc_path
FILTERED_NODELINK_SHAPE_PATH = FILTERED_PATH_CONF.metr_shapefile_path
FILTERED_ADJ_MX_PATH = FILTERED_PATH_CONF.adj_mx_path


In [ ]:
# Set up visualization parameters
plt.rcParams["font.family"] = "AppleGothic"  # Use AppleGothic for better font rendering
plt.rcParams["axes.unicode_minus"] = False  # Prevent negative sign rendering issues

## RAW 교통 데이터 특성

In [ ]:
raw = TrafficData.import_from_hdf(TRAFFIC_DATA_PATH)
df = raw.data
df.iloc[:, :5]

### 1. 데이터 범위

In [ ]:
# 데이터셋 기본 정보
def print_dataset_range_info(df: pd.DataFrame):
    num_sensors = len(df.columns)
    time_range = (df.index.min(), df.index.max())
    df_index = pd.DatetimeIndex(df.index)
    data_interval = df_index.freq or pd.infer_freq(df_index)
    total_data_points = df.size

    print(f"센서 수 (열 개수): {num_sensors:,}")
    print(f"시간 범위: {time_range[0]} ~ {time_range[1]}")
    print(f"데이터 간격: {data_interval}")
    print(f"총 데이터 포인트 (셀 개수): {total_data_points:,} (행: {len(df):,} × 열: {num_sensors:,})")

In [ ]:
print_dataset_range_info(df)

### 2. 데이터 품질

In [ ]:
def print_dataset_quality_info(df: pd.DataFrame):
    # 결측치 분석
    missing_mask = df.isna()

    # 전체 결측률
    overall_missing_rate = missing_mask.sum().sum() / df.size * 100

    # 센서별 결측률 및 결측치 개수
    sensor_missing_counts = missing_mask.sum()
    sensor_missing_rates = sensor_missing_counts / len(df) * 100

    # 센서 품질 분류
    no_missing_sensors = (sensor_missing_rates == 0).sum()
    high_quality_sensors = (sensor_missing_rates < 10).sum()
    low_quality_sensors = (sensor_missing_rates > 50).sum()
    very_low_quality_sensors = (sensor_missing_rates > 90).sum()

    # 센서별 결측치 수의 최대/최소
    max_missing_count = sensor_missing_counts.max()
    min_missing_count = sensor_missing_counts.min()

    # 결측 블록 길이 계산 (연속된 결측치 구간)
    def calculate_missing_blocks(series):
        """연속된 결측치 구간의 길이를 계산"""
        is_missing = series.isna()
        # 결측치 구간의 시작/끝 찾기
        missing_blocks = []
        in_block = False
        block_length = 0
        
        for missing in is_missing:
            if missing:
                if not in_block:
                    in_block = True
                    block_length = 1
                else:
                    block_length += 1
            else:
                if in_block:
                    missing_blocks.append(block_length)
                    in_block = False
                    block_length = 0
        
        if in_block:  # 마지막이 결측치로 끝난 경우
            missing_blocks.append(block_length)
        
        return missing_blocks

    # 모든 센서의 결측 블록 계산
    all_missing_blocks = []
    for col in df.columns:
        blocks = calculate_missing_blocks(df[col])
        all_missing_blocks.extend(blocks)

    avg_missing_block_length = sum(all_missing_blocks) / len(all_missing_blocks) if all_missing_blocks else 0
    max_missing_block_length = max(all_missing_blocks) if all_missing_blocks else 0

    # 결과 출력
    print(f"전체 결측률: {overall_missing_rate:.2f}%")
    print(f"결측치가 없는 센서 수: {no_missing_sensors:,}")
    print(f"고품질 센서 수 (결측률 < 10%): {high_quality_sensors:,}")
    print(f"저품질 센서 수 (결측률 > 50%): {low_quality_sensors:,}")
    print(f"초저품질 센서 수 (결측률 > 90%): {very_low_quality_sensors:,}")
    print(f"센서별 최대 결측치 수: {max_missing_count:,}")
    print(f"센서별 최소 결측치 수: {min_missing_count:,}")
    print(f"평균 결측 블록 길이: {avg_missing_block_length:.2f} 시간")
    print(f"최대 결측 블록 길이: {max_missing_block_length:,} 시간")

In [ ]:
print_dataset_quality_info(df)

In [ ]:
def plot_missing_rate_histogram(df: pd.DataFrame):
    # 결측치 분석
    missing_mask = df.isna()

    # 센서별 결측률 및 결측치 개수
    sensor_missing_counts = missing_mask.sum()
    sensor_missing_rates = sensor_missing_counts / len(df) * 100
    
    # 센서별 결측률 히스토그램
    plt.figure(figsize=(12, 6))
    plt.hist(sensor_missing_rates, bins=50, edgecolor='black', alpha=0.7)
    plt.xlabel('결측률 (%)', fontsize=12)
    plt.ylabel('센서 수', fontsize=12)
    plt.title('센서별 결측률 분포', fontsize=14, fontweight='bold')
    plt.grid(axis='y', alpha=0.3)

    # 품질 구간 표시
    plt.axvline(x=10, color='green', linestyle='--', linewidth=2, label='고품질 기준 (10%)')
    plt.axvline(x=50, color='orange', linestyle='--', linewidth=2, label='저품질 기준 (50%)')
    plt.axvline(x=90, color='red', linestyle='--', linewidth=2, label='초저품질 기준 (90%)')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_missing_rate_histogram(df)

### 3. 교통량 데이터 통계

In [ ]:
def print_dataset_statistics(df: pd.DataFrame):
    # 전체 데이터의 기본 통계량 (결측치 제외)
    valid_data = df.values.flatten()
    valid_data = valid_data[~pd.isna(valid_data)]

    data_min = valid_data.min()
    data_max = valid_data.max()
    data_mean = valid_data.mean()
    data_median = float(pd.Series(valid_data).median())
    data_std = valid_data.std()
    data_q25 = float(pd.Series(valid_data).quantile(0.25))
    data_q75 = float(pd.Series(valid_data).quantile(0.75))

    print("=== 전체 교통량 데이터 기본 통계량 ===")
    print(f"최소값: {data_min:.2f}")
    print(f"최대값: {data_max:.2f}")
    print(f"평균: {data_mean:.2f}")
    print(f"중앙값: {data_median:.2f}")
    print(f"표준편차: {data_std:.2f}")
    print(f"1사분위수 (25%): {data_q25:.2f}")
    print(f"3사분위수 (75%): {data_q75:.2f}")
    print(f"데이터 범위: {data_max - data_min:.2f}")
    print(f"IQR (사분위범위): {data_q75 - data_q25:.2f}")

    # 0값 비율 계산
    # 이미 데이터에서 이상 0값은 제거됨
    zero_count = (valid_data == 0).sum()
    zero_ratio = zero_count / len(valid_data) * 100
    print(f"\n0으로 기록된 측정값 비율: {zero_ratio:.2f}% ({zero_count:,} / {len(valid_data):,})")

In [ ]:
print_dataset_statistics(df)

## RAW 노드 링크 데이터 특성

In [ ]:
nodelink_raw = gpd.read_file(NODELINK_SHAPE_PATH)
nodelink_raw.head(3)

### 1. 인천시 노드링크 전체

In [ ]:
nodelink_raw.explore()

### 2. 데이터 품질을 위한 결측률 90% 미만 교통량 데이터가 있는 노드링크

In [ ]:
# 결측치 분석
missing_mask = df.isna()

# 센서별 결측률 및 결측치 개수
sensor_missing_counts = missing_mask.sum()
sensor_missing_rates = sensor_missing_counts / len(df) * 100

# 90% 미만 결측률을 가진 센서 필터링
filtered_sensors = sensor_missing_rates[sensor_missing_rates < 90].index.tolist()

print(f"전체 센서 수: {len(df.columns):,}")
print(f"90% 미만 결측률 센서 수: {len(filtered_sensors):,}")
print(f"제외된 센서 수: {len(df.columns) - len(filtered_sensors):,}")
print(f"\n필터링된 센서 리스트 (처음 10개):")
print(filtered_sensors[:10])

In [ ]:
# 필터링된 센서에 해당하는 노드 링크 데이터 추출
filtered_nodelink = nodelink_raw[nodelink_raw['LINK_ID'].isin(filtered_sensors)]

# 필터링된 데이터의 기본 정보 출력
print(f"전체 노드 링크 수: {len(nodelink_raw):,}")
print(f"필터링된 노드 링크 수: {len(filtered_nodelink):,}")
print(f"제외된 노드 링크 수: {len(nodelink_raw) - len(filtered_nodelink):,}")

# 필터링된 노드 링크 데이터 시각화
filtered_nodelink.explore()

## RAW Adjacency Matrix 분석

In [ ]:
# Adjacency Matrix 로드
with open(ADJ_MX_PATH, 'rb') as f:
    raw = pickle.load(f)
    raw_adj_mx = raw[2]

In [ ]:
def print_adj_matrix_info(adj_mx: np.ndarray, threshold: float = 0.1):
    # NetworkX 그래프로 변환 (threshold로 약한 연결 제거)
    G = nx.from_numpy_array(adj_mx)

    # 기본 통계
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    max_possible_edges = num_nodes * (num_nodes - 1) / 2
    sparsity = 1 - (num_edges / max_possible_edges)
    density = nx.density(G)

    # Connected Components 분석
    connected_components = list(nx.connected_components(G))
    num_components = len(connected_components)
    component_sizes = [len(c) for c in connected_components]
    largest_component_size = max(component_sizes)
    largest_component_ratio = largest_component_size / num_nodes * 100

    # Degree 통계
    degrees = [d for n, d in G.degree()]
    avg_degree = np.mean(degrees)
    median_degree = np.median(degrees)
    std_degree = np.std(degrees)
    max_degree = max(degrees)
    min_degree = min(degrees)

    print("=== Adjacency Matrix 그래프 분석 ===")
    print(f"\n[기본 정보]")
    print(f"노드 수: {num_nodes:,}")
    print(f"엣지 수: {num_edges:,}")
    print(f"최대 가능 엣지 수: {max_possible_edges:,.0f}")
    print(f"\n[Sparsity & Density]")
    print(f"Sparsity (희소성): {sparsity:.4f} ({sparsity*100:.2f}%)")
    print(f"Density (밀도): {density:.4f}")
    print(f"\n[Connected Components]")
    print(f"연결 그래프 개수: {num_components:,}")
    print(f"가장 큰 연결 그래프 크기: {largest_component_size:,} ({largest_component_ratio:.2f}%)")
    print(f"연결 그래프 크기 분포: {sorted(component_sizes, reverse=True)[:10]}")
    print(f"\n[Degree Statistics]")
    print(f"평균 연결도: {avg_degree:.2f}")
    print(f"중앙값 연결도: {median_degree:.2f}")
    print(f"표준편차: {std_degree:.2f}")
    print(f"최대 연결도: {max_degree}")
    print(f"최소 연결도: {min_degree}")

In [ ]:
print_adj_matrix_info(raw_adj_mx)

In [ ]:
def plot_degree_distribution(adj_mx: np.ndarray, threshold: float = 0.1):
    G = nx.from_numpy_array(adj_mx)

    degrees = [d for n, d in G.degree()]
    avg_degree = np.mean(degrees)
    median_degree = np.median(degrees)
    
    # Degree Distribution 시각화
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # 히스토그램
    axes[0].hist(degrees, bins=50, edgecolor='black', alpha=0.7)
    axes[0].axvline(avg_degree, color='red', linestyle='--', linewidth=2, label=f'평균: {avg_degree:.2f}')
    axes[0].axvline(median_degree, color='green', linestyle='--', linewidth=2, label=f'중앙값: {median_degree:.2f}')
    axes[0].set_xlabel('연결도 (Degree)', fontsize=12)
    axes[0].set_ylabel('노드 수', fontsize=12)
    axes[0].set_title('Degree Distribution', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(axis='y', alpha=0.3)

    # Log-log plot
    degree_counts = np.bincount(degrees)
    degrees_unique = np.where(degree_counts > 0)[0]
    counts = degree_counts[degrees_unique]
    axes[1].loglog(degrees_unique, counts, 'o', alpha=0.7)
    axes[1].set_xlabel('Degree (log scale)', fontsize=12)
    axes[1].set_ylabel('Count (log scale)', fontsize=12)
    axes[1].set_title('Degree Distribution (Log-Log)', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_degree_distribution(raw_adj_mx)

In [ ]:
# # 저장 경로 설정
# output_dir = "../datasets/metr-imc/subsets/v0/nodelink"
# output_path = os.path.join(output_dir, "imc_link.shp")

# # 디렉토리 생성 (존재하지 않는 경우)
# os.makedirs(output_dir, exist_ok=True)

# # Shapefile 저장
# filtered_nodelink.to_file(output_path, driver='ESRI Shapefile', encoding='utf-8')

# print(f"Shapefile 저장 완료: {output_path}")
# print(f"저장된 레코드 수: {len(filtered_nodelink):,}")

## 필터링된 데이터셋 특징

데이터셋 필터링 과정:
1. Cell 25에서 저장된 shapefile 로드
2. 영종도 및 강화도 부분 끊어진 부분 수동 제거
3. v1 서브셋 폴더 생성 및 nodelink 폴더에 저장
4. 해당 nodelink를 target으로 raw 교통량 데이터 수정
    - v1 노드링크 대상 도로만 선택
    - 2023-01-26 00:00:00 이후의 데이터만 선택 (더 깔끔한 데이터)
5. 해당 교통량 데이터를 바탕으로 데이터셋 파일 다시 생성
6. 가장 큰 연결 그래프 1개만 선택

4-5 단계는 pipeline의 generate_subset_dataset 함수로 구현되어 있음

In [ ]:
# 데이터 생성 전이라면 여기서 오류가 발생할 것으로 예상
print(FILTERED_DATA_PATH)
f_raw = TrafficData.import_from_hdf(FILTERED_DATA_PATH)
f_df = f_raw.data
f_df.iloc[:, :5].head()

In [ ]:
print(FILTERED_NODELINK_SHAPE_PATH)
f_nodelink = gpd.read_file(FILTERED_NODELINK_SHAPE_PATH)
f_nodelink.head()

In [ ]:
print(FILTERED_ADJ_MX_PATH)
with open(FILTERED_ADJ_MX_PATH, 'rb') as f:
    raw = pickle.load(f)
    f_adj_mx = raw[2]

### 1. 데이터 범위

In [ ]:
print_dataset_range_info(f_df)

### 2. 데이터 품질

In [ ]:
print_dataset_quality_info(f_df)

In [ ]:
plot_missing_rate_histogram(f_df)

### 3. 교통량 데이터 통계

In [ ]:
print_dataset_statistics(f_df)

### 4. 노드링크 시각화

In [ ]:
f_nodelink.explore()

### 5. Adjacency Matrix 분석

In [ ]:
print_adj_matrix_info(f_adj_mx)

In [ ]:
plot_degree_distribution(f_adj_mx)